# InstaCart Modeling

In this notebook, a predictive model for the Kaggle Instacart Challenge (https://www.kaggle.com/c/instacart-market-basket-analysis) is developed. Here, the challenge is, based on 3 million orders of mainly grocery products, to predict the re-ordered products of the next order of users of the Instacart service. Each order consists of none, one or multiple products, and thus the problem is a multi-label classification problem, which for each previously ordered product decides whether or not it will appear in the next order again.

The main idea of the notebook is to first generate some very basic (aggregation) features of the data explored in the EDA before. Then, we use some of these features we build a baseline model that does not involve any ML to predict the re-ordered products in an order. This yields an F1-score of 0.329 on the Kaggle public leaderboard (LB), which at the time of submission ranks in the 30th percentile of the competition.

Then, we develop a baseline classifier based on LightGBM (https://github.com/Microsoft/LightGBM) and a very basic set of features. While the very initial version has a very bad scoring of ~0.2, adding some more meaningful features boosts performance to 0.3805. Incorporating per-order thresholds increases the F1-score on the LB to 0.386, and with parameter tuning the LightGBM model we are able to further increase the score to 0.397. At the time of submission, the solution ranked in the 95th percentile of the competition (Rank 120 of ~2,500 entries).

The final model then considers a few additional features, as well as stacking multiple classifiers and merging their predicted probabilities before calculating the per-order threshold. The final ranking of the submission is in the 91st percentile.

In [11]:
######################################################
#
# Import libraries
#
######################################################

import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as sns

import collections

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold

%matplotlib inline

In [ ]:
######################################################
#
# Read in data
#
######################################################

orders = pd.read_csv('orders.csv')
prior = pd.read_csv('order_products__prior.csv')
train = pd.read_csv('order_products__train.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
products = pd.read_csv('products.csv')

# merge prior with order information
prior = pd.merge(prior, orders, on='order_id', how='left')

# Baseline Prediction (without any Machine Learning)

Here, the basic idea is to simply look at (i) the average order size of a customer and (ii) the products that that customer re-ordered most often. Then, starting from the highest re-order ratio, add products until the average order size is hit. This model results in a score of 0.329 on the Kaggle LB.

In [ ]:
test = orders[orders.eval_set == 'test']
sample_sub = pd.DataFrame(columns=['order_id', 'products'])
sample_sub['order_id'] = test['order_id']

In [ ]:
# this takes some time...
def get_products(order_id):
    prods_predicted = []
    uid = test[test['order_id'] == order_id]['user_id'].iloc[0]
    cur_prods = prods[prods['user_id'] == uid]
    avg_size = int(round(users[users['user_id'] == uid]['u_average_order_size'].iloc[0]))
    
    for x in range(0,avg_size):
        prods_predicted.append(cur_prods['product_id'].iloc[x])
    return prods_predicted


sample_sub['products'] = sample_sub['order_id'].apply(lambda x: get_products(x)).apply(lambda x: " ".join(str(y) for y in x))
sample_sub.to_csv('sample_submission.csv', index=False)

# Basic Model - Requires Basic Feature Engineering - Aggregated Stats

For our very first model, we will calculate a few basic new features. These features are mainly aggregated stats such as sums, averages, standard deviations, counts, etc. For instance, the mean of the *'reordered'* column for a product reflects the per-product reorder ratio.

There are in general three different kinds of features that we calculate now:

* User Statistics: indicated by prefix *u_*
* Product Statistics: indicated by prefix *p_*
* User x Product statistics: indicated by prefix *u_*p_

## User Stats

In [ ]:
# create dataframe for user stats
users = pd.DataFrame(prior['user_id'].unique(), columns=['user_id'])

# calculate stats based on prior data
user_stats = prior.groupby(['user_id','order_id']).size()
gb_user_id = user_stats.groupby('user_id')
prior_user_id = prior.groupby('user_id')
std_dev = gb_user_id.std().reset_index()
user_hist = gb_user_id.mean().reset_index()
order_size = prior_user_id.size().reset_index()
order_count = prior_user_id['order_number'].max().reset_index()
user_avgs = prior_user_id.mean().reset_index()
user_avgs = user_avgs[['user_id', 'reordered','order_dow', 'order_hour_of_day', 'days_since_prior_order']]

In [ ]:
# merge user frame with new features
users = pd.merge(users, std_dev, on='user_id', how='left')
users = pd.merge(users, user_hist, on='user_id', how='left')
users = pd.merge(users, order_size , on='user_id', how='left')
users = pd.merge(users, order_count , on='user_id', how='left')
users = pd.merge(users, user_avgs, on = 'user_id', how='left')

In [ ]:
# rename features to more meaningful names
users.columns = ['user_id', 'u_std_dev_order_size', 'u_average_order_size', 'u_tot_ordered_products', 
                 'u_order_count', 'u_avg_reorder_ratio', 'u_avg_order_dow', 'u_avg_order_hod', 'u_avg_dspo']

## Product Stats

In [ ]:
# make a frame for product stats
prod_stats = products.copy()

# calculate stats based on prior data
prior_by_prod = prior.groupby('product_id')
prod_avgs = prior_by_prod.mean().reset_index()
prod_count = prior[prior['reordered'] == 1].groupby('product_id').count().reset_index()
prod_size = prior_by_prod.size().reset_index()
prod_avgs = prod_avgs[['product_id', 'add_to_cart_order', 'reordered', 'order_dow', 'order_number', 
                       'order_hour_of_day', 'days_since_prior_order']]

In [ ]:
# rename features to more meaningful names
prod_avgs.columns = ['product_id', 'p_avg_add_to_cart', 'p_avg_reorder_ratio', 'p_avg_order_dow', 'p_avg_ord_num', 
                     'p_avg_order_hod', 'p_avg_dspo']
prod_count = prod_count[['product_id', 'reordered']]
prod_count.columns = ['product_id', 'p_count_reorders']
prod_size.columns = ['product_id', 'p_total_orders']

In [ ]:
# merge frames
prod_stats = pd.merge(prod_stats, prod_avgs, on='product_id', how='left')
prod_stats = pd.merge(prod_stats, prod_count, on='product_id', how='left')
prod_stats = pd.merge(prod_stats, prod_size, on='product_id', how='left')

# Product-to-User Aggregated Stats

In [ ]:
# amount of times a particular user ordered a particular item
prods = prior.groupby('user_id')['product_id'].value_counts().to_frame('times_ordered').reset_index()
# merge with some of the user stats
prods = pd.merge(prods, users[['user_id','u_order_count','u_average_order_size']], how='left', on='user_id' )

In [ ]:
# create new frame for prod_x_user stats

prods_by_users = pd.DataFrame()
prods_by_users[['user_id','product_id']] = prods[['user_id','product_id']]
prods_by_users = pd.merge(prods_by_users, prod_stats[prod_stats.columns.difference(['product_name','aisle_id','department_id'])],
                          on='product_id', how='left')

by_usr = prior.groupby(['user_id','product_id']).mean().reset_index()

In [ ]:
prods_by_users = pd.merge(prods_by_users, by_usr, on=['user_id','product_id'], how='left')
#rename columns
prods_by_users.columns = ['user_id', 'product_id', 'p_avg_add_to_cart',
       'p_avg_dspo', 'p_avg_ord_num', 'p_avg_order_dow', 'p_avg_order_hod',
       'p_avg_reorder_ratio', 'p_count_reorders', 'p_total_orders',
       'u_p_avg_order_id', 'u_p_avg_add_to_cart_order', 'u_p_avg_reordered', 'u_p_avg_order_number',
       'u_p_avg_order_dow', 'u_p_avg_order_hour_of_day', 'u_p_avg_days_since_prior_order']


In [ ]:
# some more features

prods_by_users['u_p_diff_cart_order'] = prods_by_users['p_avg_add_to_cart'] - prods_by_users['u_p_avg_add_to_cart_order']
prods_by_users['u_p_diff_days_since_prior'] = prods_by_users['p_avg_dspo'] - prods_by_users['u_p_avg_days_since_prior_order']
prods_by_users['u_p_diff_order_dow'] = prods_by_users['p_avg_order_dow'] - prods_by_users['u_p_avg_order_dow']
prods_by_users['u_p_diff_order_hod'] = prods_by_users['p_avg_order_hod'] - prods_by_users['u_p_avg_order_hour_of_day']
prods_by_users['u_p_diff_reorder_ratio'] = prods_by_users['p_avg_reorder_ratio'] - prods_by_users['u_p_avg_reordered']

prods_by_users['u_p_order_count'] = prods['times_ordered']
prods_by_users['u_p_order_ratio'] = prods['times_ordered'] / prods ['u_order_count']

# Baseline Predictor (simple classifier)

Now, we will take our model to a next step by actually involving ML algorithms in order to improve the scoring. We'll start with a rather simple ML classifier. The task at hand is multi-label classification. The approach will be: Run a multi-label classifier that yields probabilities for re-ordering, and then find a cut-off in these probabilities to determine at which point a product is projected to be re-ordered in the next order.

First, let's see what we want to predict. In the test set, we see that we are simply given an *order_id*, together with a *user_id* and several features (*order_number, order_dow, order_hour_of_day, days_since_prior_order*), and we need to predict which products the customer will re-order. For that, we first need to know which products the customer has ordered before (all other products can not be re-ordered).

Once we have a list of all products a user has ever ordered before (we can take this from the *prior* dataset), in the next step, we need to yield an estimate of whether or not each product will be re-ordered. Afterwards, we can take the predicted value to derive the set of products we expect for a particular user. 

For each product, we determine a probability of being in the next basket (i.e., a value $p$ in $[0,1]$), and then we tune a parameter $p_b$ that acts as a threshold basket probability, i.e., all items with a probability $p > p_b$ are included into the set.

In [ ]:
######################################################
#
# Prepare train data, merge with previously calculated
# stats.
#
######################################################

train = orders[orders.eval_set == 'train']
train_orders = pd.read_csv('order_products__train.csv')
train = pd.merge(train, prods, on='user_id', how='left')
train = pd.merge(train, train_orders, on=['order_id','product_id'], how='left')
train = pd.merge(train, prods_by_users, on=['user_id', 'product_id'], how='left')
train = pd.merge(train, products[['product_id','aisle_id','department_id']], on='product_id', how='left')
train = pd.merge(train, users[users.columns.difference(['u_average_order_size', 'u_order_count'])], on='user_id', how='left')
# each user has set of products he has ever ordered before.
# out of this set, only a fraction may be present in the train order,
# which means that the user has also not reordered a lot of products
# --> need to set all products not in the train order to 0
train['reordered'] = train['reordered'].fillna(0);

In [ ]:
######################################################
#
# Define features and parameters for baseline model.
# Note that we are using LightGBM as the classifier.
# LightGBM typically operates much fast than a random
# forest and has shown to achieve performance better
# or close to XGBoost.
#
######################################################

#very simple feature set, only use a couple of product related stats other than features already provided in the data
features = ['order_dow', 'order_hour_of_day', 'days_since_prior_order','p_avg_reorder_ratio', 
            'p_count_reorders', 'p_total_orders', 'aisle_id', 'department_id']

#parameters for LightGBM - need to tune these later
params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 70,
    'min_data_in_leaf': 25,
}

In [ ]:
######################################################
#
# Train LightGBM model
#
######################################################

# prepare train set
train_lgb = lgb.Dataset(train[features], train['reordered'])
res = lgb.train(params, train_lgb, 90)

In [ ]:
######################################################
#
# Prepare test data
#
######################################################

test = orders[orders.eval_set == 'test']
test = pd.merge(test, prods, on='user_id', how='left')
test = pd.merge(test, prods_by_users, on=['user_id', 'product_id'], how='left')
test = pd.merge(test, products[['product_id','aisle_id','department_id']], on='product_id', how='left')
test = pd.merge(test, users[users.columns.difference(['u_average_order_size', 'u_order_count'])], on='user_id', how='left')

In [ ]:
# for testing purposes, we don't want to mess with our test set (reusability), thus make a copy
test_cpy = test.copy()
test_cpy['preds'] = res.predict(test_cpy[features])

Now, to determine the added products: Threshold approach.

In [ ]:
test_cpy = test_cpy[['order_id', 'user_id','product_id','preds']]

######################################################
#
# Include product into set if probability is > 0.21 
# - this value was tested to yield best results, and is also ~1/2 of the F1 score leading the LB.
# This has been shown to be a good approximation of the optimal threshold, see: https://arxiv.org/abs/1402.1892
#
######################################################

test_cpy['included'] = test_cpy['preds'].apply(lambda x: 1 if x > 0.21 else 0)
test_cpy = test_cpy[test_cpy.included == 1]

In [ ]:
def get_products(order_id):
    subset = test_cpy[test_cpy['order_id'] == order_id]
    
    if len(subset) == 0:
        return 'None'
    else:
        prods_predicted = []
        prods_predicted.extend(subset['product_id'])
        return prods_predicted


sample_sub = pd.DataFrame(columns=['order_id', 'products'])
sample_sub['order_id'] = test['order_id'].unique()
sample_sub['products'] = sample_sub['order_id'].apply(lambda x: get_products(x)).apply(lambda x: (" ".join(str(y) for y in x) if x != 'None' else 'None'))

sample_sub = sample_sub.sort_values(by='order_id')
sample_sub.to_csv('sample_submission_baseline_thresh.csv', index=False)

This gives a result of just 0.20 on the Kaggle LB. However, this is to be expected, since there are just very few meaningful features that would help to obtain a good score - in other words, the probabilities of inclusion are too similar for most products and the information retained in the baseline (without ML) is more helpful here. 

# Improving the Baseline Classifier

We now want to improve the performance of the baseline classifier by adding meaningful features. To evaluate whether or not a feature gives us some improvement, we will use cross-validation. We should not use random allocation of these sets in order to avoid overfitting to particular users. Rather, train and validation should not have overlapping users, and subsequently, the cross-validation folds used in the train set should respect the same property

In [ ]:
train_lgb = lgb.Dataset(train[features], train['reordered'], categorical_feature=['aisle_id', 'department_id',
                                                                                  'order_dow','order_hour_of_day',
                                                                                  'days_since_prior_order'])

#cast LGB to sklearn (better cross validation functionality)
clf = lgb.LGBMClassifier(
    task = 'train',
    boosting_type = 'dart',
    objective = 'binary',
    metric = {'binary_logloss'},
    num_leaves = 70,
    min_data_in_leaf = 25)

In [ ]:
# Define custom folds based on unique user_ids
custom_folds = GroupKFold(n_splits=5).split(X=train[train.columns.difference(['reordered'])], 
                                            y=train['reordered'], 
                                            groups=train['user_id'])

# Compute cross validation score  
cross_val_score(clf, 
                X=train[features], 
                y=train['reordered'],
                scoring='neg_log_loss',
                cv=custom_folds)

CV Log-loss is currently ~0.393 - Now we can add additional features to check on how to improve it.

In [ ]:
#all of the following features contribute to improving the score, i.e., reducing log-loss
features = [ 
            # general features
            'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'aisle_id', 'department_id',
            # user related features
            'u_tot_ordered_products', 'u_order_count', 'u_avg_reorder_ratio', 
            # product-related features
            'p_avg_reorder_ratio', 'p_count_reorders', 'p_total_orders', 'p_avg_add_to_cart',
            # user-product-features
            'u_p_avg_reordered', 'u_p_avg_order_number', 'u_p_order_ratio'
           ]

train_lgb = lgb.Dataset(train[features], train['reordered'], 
                        categorical_feature=['aisle_id', 'department_id','order_dow','order_hour_of_day',
                                             'days_since_prior_order'])

# Define custom folds
custom_folds = GroupKFold(n_splits=5).split(X=train[train.columns.difference(['reordered'])], 
                                            y=train['reordered'], 
                                            groups=train['user_id'])

cross_val_score(clf, 
                X=train[features], 
                y=train['reordered'],
                scoring='neg_log_loss',
                cv=custom_folds)


Now, log-loss is below 0.37. Let's try to use these features as our next baseline and train the model on the entire train set.

In [ ]:
res = lgb.train(params, train_lgb, 90)

# let's see which features were most important
lgb.plot_importance(res, figsize=(12,10))

In [ ]:
# repeat as previously
test_cpy = test.copy()
test_cpy['preds'] = res.predict(test_cpy[features])
test_cpy = test_cpy[['order_id', 'user_id','product_id','u_average_order_size','preds']]
test_cpy['included'] = test_cpy['preds'].apply(lambda x: 1 if x > 0.21 else 0)
test_cpy = test_cpy[test_cpy.included == 1]

In [ ]:
# this takes some time...
def get_products(order_id):
    subset = test_cpy[test_cpy['order_id'] == order_id]
    
    if len(subset) == 0:
        return 'None'
    else:
        prods_predicted = []
        prods_predicted.extend(subset['product_id'])
        return prods_predicted


sample_sub = pd.DataFrame(columns=['order_id', 'products'])
sample_sub['order_id'] = test['order_id'].unique()
sample_sub['products'] = sample_sub['order_id'].apply(lambda x: get_products(x)).apply(lambda x: (" ".join(str(y) for y in x) if x != 'None' else 'None'))

sample_sub = sample_sub.sort_values(by='order_id')
sample_sub.to_csv('sample_submission_agg_features.csv', index=False)

This scores 0.371 on the LB, which is a huge improvement, not only towards the baseline lightGBM classifier, but also to the baseline without ML.

# Advanced feature engineering

Previously, our features are very basic statistics that can be derived by pandas aggregation functionalities. We will now introduce additional features, which will often be derived from existing features in order to determine better which product is valuable to which user, and thus more likely to be re-ordered. Examples for such features are:

* Was the product contained in the last order? Was it in the second last order? Was it in the 3rd last order? All users have at least three recorded orders in the prior set, so we can check this. The assumption is that more recent orders tell us more about what items a customer will reorder.
* How does the current order differ from previous ones (hour of day, day of week, days since prior order difference)?
* How much time has passed since the user ordered the product the last time?
* How much time has passed since the user ordered the product the first time?
* etc.

Additionally, we will have some more rather basic features (like the number of distinct products a user has ordered, which may hint at how consistent he/she is).

In [ ]:
######################################################
#
# All of the following features reduce logloss. Note 
# that many more features have been tried, but are not
# included as they did not help to improve logloss,
# and sometimes worsened it
#
######################################################
train['ratio_dspo'] = train['days_since_prior_order'] / train['u_p_avg_days_since_prior_order']
train['diff_order_dow'] = train['order_dow'] - train['u_p_avg_order_dow']
train['diff_order_hod'] = train['order_hour_of_day'] - train['u_p_avg_order_hour_of_day']
train['max_dspo_order'] = train['days_since_prior_order'].apply(lambda x: 1 if x==30 else 0)

In [ ]:
# difference in dspo
orders['o_dspo_diff'] = orders['days_since_prior_order'].diff().fillna(orders['days_since_prior_order'])

# lifetime of a user in the system, the cumulative sum of days since prior order
orders['o_user_lifetime'] = orders.groupby('user_id').cumsum()['days_since_prior_order']

# number of distinct products ordered by the user
dist_prods =  prior.groupby('user_id')['product_id'].nunique().reset_index()
dist_prods.columns = ['user_id', 'u_dist_prods']

# merge dfs
users = pd.merge(users, dist_prods, on='user_id', how='left')
train = pd.merge(train, orders[['order_id','o_dspo_diff', 'o_user_lifetime']], on='order_id', how='left')
train = pd.merge(train, users[['user_id','u_dist_prods']], on='user_id', how='left')

In [ ]:
#get max order number 
tot_orders = users[['user_id','u_order_count']]
orders = pd.merge(orders, tot_orders, on='user_id', how='left')
prior = pd.merge(prior, orders[['order_id','u_order_count']], on='order_id', how='left')

In [ ]:
# list all products that were in the last order
last_order_products = prior[prior['order_number'] == prior['u_order_count']].groupby(['user_id'])['product_id'].aggregate(lambda x: list(x)).reset_index()
last_order_products.columns = ['user_id','last_order_prods']
train = pd.merge(train, last_order_products, on='user_id', how='left')
# for each product in the train set (i.e., each product the user has ever ordered), check whether it was in the last order
train['u_p_in_last_order'] = train.apply(lambda x: 1 if x['product_id'] in x['last_order_prods'] else 0, axis=1)

In [ ]:
# repeat above steps also for 2nd, and 3rd last order 
# - each user has at least 3 orders recorded, so we can safely go back that far
two_orders_ago = prior[prior['order_number'] == (prior['u_order_count']-1)].groupby(['user_id'])['product_id'].aggregate(lambda x: list(x)).reset_index()
three_orders_ago = prior[prior['order_number'] == (prior['u_order_count']-2)].groupby(['user_id'])['product_id'].aggregate(lambda x: list(x)).reset_index()
two_orders_ago.columns = ['user_id','2nd_last_order_prods']
three_orders_ago.columns = ['user_id','3rd_last_order_prods']
train = pd.merge(train, two_orders_ago, on='user_id', how='left')
train = pd.merge(train, three_orders_ago, on='user_id', how='left')
train['u_p_in_2nd_last_order'] = train.apply(lambda x: 1 if x['product_id'] in x['2nd_last_order_prods'] else 0, axis=1)
train['u_p_in_3rd_last_order'] = train.apply(lambda x: 1 if x['product_id'] in x['3rd_last_order_prods'] else 0, axis=1)

In [ ]:
# min/max aggregations - this takes a long time, execute with care!
u_p_max = prior.groupby(['user_id','product_id']).max().reset_index()
u_p_min = prior.groupby(['user_id','product_id']).min().reset_index()

In [ ]:
u_p_max.columns = ['user_id', 'product_id', 'u_p_max_order_id', 'u_p_max_add_to_cart_order', 'u_p_max_reordered',
                   'u_p_max_eval_set', 'u_p_max_order_number', 'u_p_max_order_dow', 'u_p_max_order_hour_of_day', 
                   'u_p_max_days_since_prior_order', 'u_p_max_order_count']
u_p_min.columns = ['user_id', 'product_id', 'u_p_min_order_id', 'u_p_min_add_to_cart_order', 'u_p_min_reordered',
                   'u_p_min_eval_set', 'u_p_min_order_number', 'u_p_min_order_dow', 'u_p_min_order_hour_of_day', 
                   'u_p_min_days_since_prior_order', 'u_p_min_order_count']

In [ ]:
train = pd.merge(train, u_p_max[['user_id', 'product_id', 'u_p_max_order_id', 'u_p_max_add_to_cart_order', 'u_p_max_reordered',
                   'u_p_max_order_number', 'u_p_max_order_dow', 'u_p_max_order_hour_of_day', 
                   'u_p_max_days_since_prior_order', 'u_p_max_order_count']],
                 on=['user_id', 'product_id'], how='left')
train = pd.merge(train, u_p_min[['user_id', 'product_id', 'u_p_min_order_id', 'u_p_min_add_to_cart_order', 'u_p_min_reordered',
                   'u_p_min_order_number', 'u_p_min_order_dow', 'u_p_min_order_hour_of_day', 
                   'u_p_min_days_since_prior_order', 'u_p_min_order_count']],
                 on=['user_id', 'product_id'], how='left')

In [ ]:
# determine when the user has first bought the product, and when he has last ordered it
train['u_p_time_since_last_buy'] = train['order_number'] - train['u_p_max_order_number']
train['u_p_time_since_first_buy'] = train['order_number'] - train['u_p_min_order_number']

Time to test whether our features are actually working or not! Note that the below list of features is rather small compared to the features we have created in the course of this notebook. All features that are not listed below do not yield any improvement on the CV score and were thus not included in the model.

In [ ]:
#all of the following features contribute to decreasing log loss
 
features = [ 
            # BASIC general features
            'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'aisle_id', 'department_id',
            # BASIC user related features
            'u_tot_ordered_products', 'u_order_count', 'u_avg_reorder_ratio', 'u_average_order_size',
            'u_dist_prods',
            # BASIC product-related features
            'p_avg_reorder_ratio', 'p_count_reorders', 'p_total_orders', 'p_avg_add_to_cart',
            # BASIC user-product-features
            'u_p_avg_reordered', 'u_p_avg_order_number', 'u_p_order_ratio', 'u_p_max_order_number',
            'u_p_min_order_number',           
            # ratios
            'ratio_dspo', 
            # differences
            'diff_order_dow', 'diff_order_hod', 'o_dspo_diff', 'u_p_time_since_last_buy', 
            'u_p_time_since_first_buy',
            # categoricals
            'max_dspo_order',  'u_p_in_last_order', 'u_p_in_2nd_last_order', 'u_p_in_3rd_last_order', 
           ]


train_lgb = lgb.Dataset(train[features], train['reordered'])

# Define custom folds
custom_folds = GroupKFold(n_splits=5).split(X=train[train.columns.difference(['reordered'])], 
                                            y=train['reordered'], 
                                            groups=train['user_id'])

clf = lgb.LGBMClassifier(
    task = 'train',
    boosting_type = 'dart',
    objective = 'binary',
    metric = {'binary_logloss'},
    num_leaves = 70,
    min_data_in_leaf = 25)

cross_val_score(clf, 
                X=train[features], 
                y=train['reordered'],
                scoring='neg_log_loss',
                cv=custom_folds)

array([-0.36546325, -0.3658442 , -0.36593602, -0.36538184, -0.36470168]) # for outputcheck


A tiny bit improvement in the score! Log-loss @ 0.365 - now we will mirror these features to the test set, and then subsequently train the model again to check the score on the leaderboard.

In [ ]:
# mirror features in test set
test['ratio_dspo'] = test['days_since_prior_order'] / test['u_p_avg_days_since_prior_order']
test['diff_order_dow'] = test['order_dow'] - test['u_p_avg_order_dow']
test['diff_order_hod'] = test['order_hour_of_day'] - test['u_p_avg_order_hour_of_day']
test['max_dspo_order'] = test['days_since_prior_order'].apply(lambda x: 1 if x==30 else 0)

In [ ]:
test = pd.merge(test, orders[['order_id','o_dspo_diff', 'o_user_lifetime']], on='order_id', how='left')
test = pd.merge(test, users[['user_id','u_dist_prods']], on='user_id', how='left')

In [ ]:
test = pd.merge(test, u_p_max[['user_id', 'product_id', 'u_p_max_order_id', 'u_p_max_add_to_cart_order', 'u_p_max_reordered',
                   'u_p_max_order_number', 'u_p_max_order_dow', 'u_p_max_order_hour_of_day', 
                   'u_p_max_days_since_prior_order', 'u_p_max_order_count']],
                 on=['user_id', 'product_id'], how='left')
test = pd.merge(test, u_p_min[['user_id', 'product_id', 'u_p_min_order_id', 'u_p_min_add_to_cart_order', 'u_p_min_reordered',
                   'u_p_min_order_number', 'u_p_min_order_dow', 'u_p_min_order_hour_of_day', 
                   'u_p_min_days_since_prior_order', 'u_p_min_order_count']],
                 on=['user_id', 'product_id'], how='left')
test['u_p_time_since_last_buy'] = test['order_number'] - test['u_p_max_order_number']
test['u_p_time_since_first_buy'] = test['order_number'] - test['u_p_min_order_number']

In [ ]:
test = pd.merge(test, last_order_products, on='user_id', how='left')
test['u_p_in_last_order'] = test.apply(lambda x: 1 if x['product_id'] in x['last_order_prods'] else 0, axis=1)

In [ ]:
test = pd.merge(test, two_orders_ago, on='user_id', how='left')
test = pd.merge(test, three_orders_ago, on='user_id', how='left')
test['u_p_in_2nd_last_order'] = test.apply(lambda x: 1 if x['product_id'] in x['2nd_last_order_prods'] else 0, axis=1)
test['u_p_in_3rd_last_order'] = test.apply(lambda x: 1 if x['product_id'] in x['3rd_last_order_prods'] else 0, axis=1)

Train the model and submit scores!

In [ ]:
# Note: LightGBM params are an initial guess - we will tune them later on
params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 70,
    'min_data_in_leaf': 25,
}

res = lgb.train(params, train_lgb, 90)
lgb.plot_importance(res, figsize=(12,10))

In [ ]:
test_cpy = test.copy()
test_cpy['preds'] = res.predict(test_cpy[features])
test_cpy = test_cpy[['order_id', 'user_id','product_id','u_average_order_size','preds']]
test_cpy['included'] = test_cpy['preds'].apply(lambda x: 1 if x > 0.21 else 0)
test_cpy = test_cpy[test_cpy.included == 1]

In [ ]:
def get_products(order_id):
    subset = test_cpy[test_cpy['order_id'] == order_id]
    
    if len(subset) == 0:
        return 'None'
    else:
        prods_predicted = []
        prods_predicted.extend(subset['product_id'])
        return prods_predicted


sample_sub = pd.DataFrame(columns=['order_id', 'products'])
sample_sub['order_id'] = test['order_id'].unique()
sample_sub['products'] = sample_sub['order_id'].apply(lambda x: get_products(x)).apply(lambda x: (" ".join(str(y) for y in x) if x != 'None' else 'None'))

sample_sub = sample_sub.sort_values(by='order_id')
sample_sub.to_csv('sample_submission_agg_features_test.csv', index=False)

In [ ]:
######################################################
#
# Save current dfs to disk so that we do not have to
# re-calculate features again.
# Storing in a binary format (here: hdf) prevents
# conversion of float to decimals, and thus keeps the
# original data.
#
######################################################

train.to_hdf('train.hdf','train_data')
test.to_hdf('test.hdf', 'test_data')

This solutions boosts the leaderboard score up to 0.3805. Again, a considerable boost. The process so far was rather simple and consisted purely of feature engineering. Let's check out something more interesting.

# Improving Performance with expected F1 score and handling None

So far, our approach was built on the assumption that there is one general threshold that is fitting well across all orders from all customers. Of course, in reality, users are very different from each other in their shopping behaviour. Some may be rather consistent, while others can have pretty wide-spread product orders with a high variance in their orders.

Therefore, it would be better to obtain *order-dependant* thresholds. There are a couple of ways to achieve this. We could, for instance, predict the average basket size of a user, and then select the products with the highest probability to go into the basket. 

This would require a separate classifier, which, based on features like average user basket size, variance in basket sizes, basket sizes in dependance of hour of day or day of week, etc., could try to predict a proper basket size.

Another alternative is to calculate the per-order threshold as a function of the expected F1 score. For details, see the paper: "Ye, N., Chai, K., Lee, W., and Chieu, H.  Optimizing F-measures: A Tale of Two Approaches.", in Proceedings of the International Conference on Machine Learning, 2012.
Basically, what this script does is to look at the probabilities obtained by our classifier and then finds a per-order threshold based on these probabilities. The threshold $t$ is found by looking at which threshold maximizes the expected F1 score for the set of probabilities. Finally, the script returns the basket size (k).

Also, we will handle the case in which an order is not containing any reordered product. If this is the case, we need to predict an explicit 'None'. Our approach here is to also encode this scenario as a probability. In particular: $p_i(None) = 1 - p_i(prod_1) * p_i(prod_2)* ... * p_i(prod_j)$ for each user $i$ with $j$ product candidates. Then, if $p_i(None) > t$, it is included into the prediction, if not, it is not. Note that this can lead to situations, in which both 'None' and (a set of) products can be ultimately predicted.

In [17]:
from f1_maximizer import F1Optimizer as f1opt

In [ ]:
test_cpy = test.copy()
test_cpy['preds'] = res.predict(test_cpy[features])
test_cpy = test_cpy[['order_id', 'user_id','product_id','preds']]

In [ ]:
test_cpy = test_cpy.sort_values(by='preds', ascending=False)
test_cpy['prod_prob'] = zip(test_cpy['product_id'], test_cpy['preds'])
prod_probs_by_order = test_cpy.groupby('order_id')['prod_prob'].aggregate(lambda x: list(x)).reset_index()
prod_probs_by_order.columns = ['order_id','prod_probs']

In [ ]:
#this takes a long time, but does the job - @TODO: get rid of for loop
final_pred = {}
for order_id in prod_probs_by_order['order_id']:
    pred_prods = []
    unzipped = zip(*prod_probs_by_order[prod_probs_by_order['order_id'] == order_id]['prod_probs'].values[0])
    k, none_in, exp = f1opt.maximize_expectation(unzipped[1])       
    pred_prods.extend(unzipped[0][:k])
    if (none_in) or (k == 0):
        pred_prods.append('None')  
    final_pred[order_id] = pred_prods

In [ ]:
sample_sub = pd.DataFrame(columns=['order_id', 'products'])
f_pred = collections.OrderedDict(sorted(final_pred.items()))
sample_sub['order_id'] = f_pred.keys()
sample_sub['products'] = f_pred.values()

In [ ]:
sample_sub['products'] = sample_sub['products'].apply(lambda x: (" ".join(str(y) for y in x)))


In [ ]:
sample_sub.to_csv('sample_submission_agg_features_test.csv', index=False)

This further increases the model performance to a leaderboard score of 0.386!

# Improving Performance by Parameter Tuning

Next, we further try to increase the performance by parameter tuning. Here, we have to consider the basic trade-off of model complexity: The more complex a model is, the more likely it is to overfit on the training data. Hence, cross validation is extremely important here.

Below is a final definition of the parameters we will use. Commented beyond that are parameter settings that have been tried out in the process. The most gain is achived with, in decreasing importance:

* learning_rate: tuning this parameter yields a huge performance boost on the local log-loss CV score.
* num_leaves
* boosting_type: dart yields a .01 improvement over gbdt.

We observe that the local cv score has *drastically* decreased to ~.245. Parameter tuning boosts our score to 0.397.

In [ ]:
features = [ 
            # BASIC general features
            'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'aisle_id', 'department_id',
            # BASIC user related features
            'u_tot_ordered_products', 'u_order_count', 'u_avg_reorder_ratio', 'u_average_order_size',
            'u_dist_prods',
            # BASIC product-related features
            'p_avg_reorder_ratio', 'p_count_reorders', 'p_total_orders', 'p_avg_add_to_cart',
            # BASIC user-product-features
            'u_p_avg_reordered', 'u_p_avg_order_number', 'u_p_order_ratio', 'u_p_max_order_number',
            'u_p_min_order_number',           
            # ratios
            'ratio_dspo', 
            # differences
            'diff_order_dow', 'diff_order_hod', 'o_dspo_diff', 'u_p_time_since_last_buy', 
            'u_p_time_since_first_buy',
            # categoricals
            'max_dspo_order',  'u_p_in_last_order', 'u_p_in_2nd_last_order', 'u_p_in_3rd_last_order', 
           ]

clf = lgb.LGBMClassifier(
    task = 'train',
    boosting_type = 'dart', # tried: [gbdt, dart, goss] 
    objective = 'binary',
    metric = {'binary_logloss'},
    num_leaves = 256, # tried: [40, 60, 80, 100, 128, 192, 225, 240, 256, 270, 300]
    learning_rate = 0.55, # tried: [0.02, 0.05, 0.1, 0.2, 0.3, 0.5, 0.52, 0.54, 0.55, 0.56, 0.6, 0.7] (seems very high!)
    min_data_in_leaf = 25, # tried: [10, 20, 22, 24, 25, 26, 27, 30]
    )


custom_folds = GroupKFold(n_splits=5).split(X=train[features], 
                                            y=train['reordered'], 
                                            groups=train['user_id'])


cross_val_score(clf, 
                X=train[features], 
                y=train['reordered'],
                scoring='neg_log_loss',
                cv=custom_folds)

array([-0.24477021, -0.24627809, -0.24580046, -0.2452874 , -0.24366409])
(Outputcheck)

In [17]:
######################################################
#
# Final training of model!
#
######################################################

train_lgb = lgb.Dataset(train[features], list(train['reordered'].values))
res = lgb.train(params, train_lgb, 2000)


test_cpy = test.copy()
test_cpy['preds'] = res.predict(test_cpy[features])
test_cpy = test_cpy[['order_id', 'user_id','product_id','preds']]

test_cpy = test_cpy.sort_values(by='preds', ascending=False)
test_cpy['prod_prob'] = zip(test_cpy['product_id'], test_cpy['preds'])
prod_probs_by_order = test_cpy.groupby('order_id')['prod_prob'].aggregate(lambda x: list(x)).reset_index()
prod_probs_by_order.columns = ['order_id','prod_probs']

#this takes a long time, but does the job - @TODO: get rid of for loop
final_pred = {}
for order_id in prod_probs_by_order['order_id']:
    pred_prods = []
    unzipped = zip(*prod_probs_by_order[prod_probs_by_order['order_id'] == order_id]['prod_probs'].values[0])
    k, none_in, exp = f1opt.maximize_expectation(unzipped[1])       
    pred_prods.extend(unzipped[0][:k])
    if (none_in) or (k == 0):
        pred_prods.append('None')  
    final_pred[order_id] = pred_prods
    

f_pred = collections.OrderedDict(sorted(final_pred.items()))
sample_sub['order_id'] = f_pred.keys()
sample_sub['products'] = f_pred.values()
sample_sub['products'] = sample_sub['products'].apply(lambda x: (" ".join(str(y) for y in x)))
sample_sub.to_csv('submission_all_features_params_own.csv', index=False)

### Finally: some notes on applying model stacking in this competition

The final boost on the Kaggle LB was obtained by stacking this model with results obtained by using XGBoost and RandomForest classifiers. The stacking method was a simple median selection, i.e., the probability fed into the f1 optimizer for selecting an item was the median probability retrieved from the output of the three models. This helped in catching some misclassifications.

Stacking did not have a large impact. The boost for the final solution was from 0.397 to 0.403 (i.e., a boost of 0.006). This lifted this solution to within ~1% of the winning solution.

Note however that the trade-off of introducing a training cost of more than 3x (XGBoost and particularly RandomForest are much more computationally expensive than LightGBM) versus a marginal predictive gain is probably not worth the effort, should we use our system in practice. It is therefore left out of this document.